In [1]:
import gc
import math
import os
import pathlib
import time

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm_notebook

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, train_test_split

import lightgbm as lgb
import xgboost as xgb

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import adam
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
%matplotlib inline

In [293]:
def add_features(
        df,
        first_index=None,
        last_index=None,
        sample_size=150000,
        holdout_size=50000,
        smootch_windows_size = (3, 5, 7)
    ):
    
    if first_index == None or last_index == None:
        first_index = df.index.min()
        last_index = df.index.max()
    sample_indexes = np.random.randint(first_index, last_index, sample_size)
    sample_indexes.sort()
    print("sample_indexes.shape:", sample_indexes.shape)    
    smootch_feature_names = ['smootch_mean_ws_{}'.format(window_size) for window_size in smootch_windows_size]
    #half_windows_size = [ws // 2 for ws in smootch_windows_size]

    acoustic_data_series = df['acoustic_data']
    
    sample_df = df.iloc[sample_indexes]
    sample_df.reset_index(inplace=True)
    sample_df.drop(columns=['index'], inplace=True)
    print("just after create sample_df, sample_df.shape[0]:", sample_df.shape[0])
    
    #for feature_name in smootch_feature_names:
    #    sample_df[feature_name] = 0
    begin_smootch_features_value = []
    end_smootch_features_value = []
    
    #sample_indexes_set = set(sample_indexes)
    begin_indexes_set = set()
    end_indexes_set = set()
    
    start_time = time.time()
    print("sample_df.shape[0] just before main loop:", sample_df.shape[0])
    sample_df_len = sample_df.shape[0]
    for window_size, feature_name in zip(smootch_windows_size, smootch_feature_names):
        
        feature_values_list = list(range(sample_size))
        
        half_window_size = window_size // 2
        begin_indexes = sample_indexes[:half_window_size]        
        sample_begin_indexes = sample_indexes[:half_window_size]
        full_data_begin_indexes = df.index[:half_window_size].tolist()

        in_window_begin_indexes = [idx for idx in sample_begin_indexes if idx in full_data_begin_indexes]
        
        sample_end_indexes = sample_indexes[-half_window_size:]

        full_data_end_indexes = df.index[-half_window_size:].tolist()
        
        in_window_end_indexes = [idx for idx in sample_end_indexes if idx in full_data_end_indexes]
        begin_indexes = begin_indexes[begin_indexes <= half_window_size]
        
        end_indexes = sample_indexes[-half_window_size:]
        
        if in_window_begin_indexes:
            begin_indexes_set.union(set(in_window_begin_indexes))
            for i, b_idx in enumerate(in_window_begin_indexes):
                #sample_df[feature_name].iloc[b_idx] = df[b_idx]['acoustic_data'] - df.iloc[first_index:first_index + window_size]['acoustic_data'].mean()
                feature_values_list[i] = df[b_idx]['acoustic_data'] - df.iloc[first_index:first_index + window_size]['acoustic_data'].mean()
        if in_window_end_indexes:
            end_indexes_set.union(set(in_window_end_indexes))
            for i, e_idx in enumerat(in_window_end_indexes):
                #sample_df[feature_name].iloc[e_idx] = df[e_idx]['acoustic_data'] - df.iloc[last_index - window_size:]['acoustic_data'].mean()
                feature_values_list[-i] = df[e_idx]['acoustic_data'] - df.iloc[last_index - window_size:]['acoustic_data'].mean()
        slice_begin = len(begin_indexes_set)
        slice_end = sample_df_len - len(end_indexes_set)
        for i in range(sample_df_len)[slice_begin:slice_end]:
            sample_idx = sample_indexes[i]
            feature_values_list[i] = acoustic_data_series.iloc[sample_idx - half_window_size:sample_idx + half_window_size].mean()
        print("sample_df.shape[0] before assign feature_values_list:", sample_df.shape[0])
        print("len(feature_values_list):", len(feature_values_list))
        sample_df[feature_name] = feature_values_list
        holdout_indexes = np.random.randint(0, sample_df.shape[0], holdout_size)
        print("sample_df.index:\n", sample_df.index.tolist())
        print("holdout_indexes:\n", holdout_indexes)
        #train_indexes = np.array(sample_df.index)
        holdout_df = sample_df.iloc[holdout_indexes]
        holdout_df.reset_index(inplace=True)
        holdout_df.drop(columns=['index'], inplace=True)
        sample_df_indexes_set = set(sample_df.index)
        train_indexes = sorted(tuple(sample_df_indexes_set.difference(set(holdout_indexes))))
        print("train_indexes:\n", train_indexes)
        sample_df = sample_df.iloc[train_indexes]
        sample_df.reset_index(inplace=True)
        sample_df.drop(columns=['index'], inplace=True)
    print("Full calculation feature value time (with slicing) {} min:".format((time.time() - start_time) / 60))
    return sample_df, holdout_df

In [184]:
earthquake_margin_indexes =[
    5656573,
    50085877,
    104677355,
    138772452,
    187641819,
    218652629,
    245829584,
    307838916,
    338276286,
    375377847,
    419368879,
    461811622,
    495800224,
    528777114,
    585568143,
    621985672
]

In [8]:
earthquakes_length = [earthquake_margin_indexes[i + 1] - earthquake_margin_indexes[i] for i in range(len(earthquake_margin_indexes) - 1)]

In [9]:
earthquakes_length

[44429304,
 54591478,
 34095097,
 48869367,
 31010810,
 27176955,
 62009332,
 30437370,
 37101561,
 43991032,
 42442743,
 33988602,
 32976890,
 56791029,
 36417529]

In [32]:
complete_earthquakes_length = earthquakes_length[:-1]

In [15]:
#complete_earthquaces_length = complete_earthquaces_length[:-1]

In [33]:
complete_earthquakes_length

[44429304,
 54591478,
 34095097,
 48869367,
 31010810,
 27176955,
 62009332,
 30437370,
 37101561,
 43991032,
 42442743,
 33988602,
 32976890,
 56791029]

In [92]:
%time
earthquake_1_df = pd.read_csv(
    '../input/train/train.csv',
    #nrows=100000000,
    names=['acoustic_data', 'time_to_failure'],
    dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32},
    skiprows=earthquake_margin_indexes[0],
    nrows=complete_earthquakes_length[0]
)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.68 µs


In [49]:
earthquake_1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44429304 entries, 0 to 44429303
Data columns (total 2 columns):
acoustic_data      float32
time_to_failure    float32
dtypes: float32(2)
memory usage: 339.0 MB


%%time
earthquake_1_with_additional_features_df = features_maker(earthquake_1_df)

In [56]:
#np.random.randint?

In [296]:
%%time
earthquake_1_with_additional_features_df, holdout_df = add_features(earthquake_1_df, sample_size=1000, holdout_size=200)

sample_indexes.shape: (1000,)
just after create sample_df, sample_df.shape[0]: 1000
sample_df.shape[0] just before main loop: 1000
sample_df.shape[0] before assign feature_values_list: 1000
len(feature_values_list): 1000
sample_df.index:
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 

sample_df.shape[0] before assign feature_values_list: 821
len(feature_values_list): 1000


ValueError: Length of values does not match length of index

In [266]:
print(earthquake_1_with_additional_features_df[:10])
print()
print(earthquake_1_with_additional_features_df[-10:])

   acoustic_data  time_to_failure  smootch_mean_ws_3  smootch_mean_ws_5  \
0            5.0        11.540800                7.0               5.75   
1            9.0        11.540800                9.0               7.00   
2            7.0        11.540800                7.0               8.00   
3            4.0        11.540800                5.5               5.50   
4            9.0        11.540800                7.5               6.50   
5            5.0        11.540800                5.0               4.25   
6            5.0        11.540800                3.5               3.00   
7            6.0        11.540800                7.5               7.00   
8            5.0        11.540800                5.5               6.25   
9            3.0        11.540799                3.5               4.00   

   smootch_mean_ws_7  
0           4.500000  
1           6.500000  
2           8.000000  
3           4.833333  
4           5.500000  
5           3.333333  
6           2